In [15]:
import os

commands = [
    'pip install protobuf',
    # 'pip install gtfs_realtime_pb2',
    'apt update -y',
    'apt install protobuf-compiler -y',
]

[os.system(cmd) for cmd in commands];


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip




Hit:1 http://deb.debian.org/debian bookworm InRelease
Hit:2 http://deb.debian.org/debian bookworm-updates InRelease
Hit:3 http://deb.debian.org/debian-security bookworm-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
9 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...


Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.21.12-3).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [15]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict  # Importar la función MessageToDict
import json

# Ruta al archivo .proto
nombre_de_archivo_proto = '/app/data/input/proto_file/input.proto'

# Cargar el archivo .proto
feed = gtfs_realtime_pb2.FeedMessage()
with open(nombre_de_archivo_proto, 'rb') as proto_file:
    feed.ParseFromString(proto_file.read())

# Listas para almacenar las entidades con y sin 'trip_update'
entities_with_trip_update = []
entities_without_trip_update = []

# Dividir las entidades en las dos listas basadas en la presencia del campo 'trip_update'
for entity in feed.entity:
    entity_dict = MessageToDict(entity)  # Convertir el objeto FeedEntity a un diccionario
    if entity.HasField('trip_update'):
        entities_with_trip_update.append(entity_dict)
    else:
        entities_without_trip_update.append(entity_dict)

# Guardar las entidades en archivos .json separados
with open("/app/data/output/json_file/with_trip_update.json", "w") as file:
    json.dump({"entity": entities_with_trip_update}, file)

with open("/app/data/output/json_file/without_trip_update.json", "w") as file:
    json.dump({"entity": entities_without_trip_update}, file)


In [20]:
import pandas as pd
import json

def load_with_trip_update(json_path):
    """
    Carga el archivo JSON con actualizaciones de viaje y lo convierte en un DataFrame.
    
    Parámetros:
    - json_path (str): Ruta del archivo JSON a cargar.
    
    Retorna:
    - DataFrame: Datos del archivo JSON en formato tabular.
    """
    with open(json_path, "r") as file:
        data = json.load(file)
        
    df = pd.json_normalize(data=data['entity'], 
                           record_path=['tripUpdate', 'stopTimeUpdate'], 
                           meta=[['tripUpdate', 'trip', 'tripId'],
                                 ['tripUpdate', 'trip', 'startTime'],
                                 ['tripUpdate', 'trip', 'startDate'],
                                 ['tripUpdate', 'trip', 'routeId'],
                                 ['tripUpdate', 'trip', 'directionId'],
                                 ['tripUpdate', 'vehicle', 'licensePlate'],
                                 ['tripUpdate', 'timestamp'],
                                 'id'],
                           errors='ignore')
    return df

def load_without_trip_update(json_path):
    """
    Carga el archivo JSON sin actualizaciones de viaje y lo convierte en un DataFrame.
    
    Parámetros:
    - json_path (str): Ruta del archivo JSON a cargar.
    
    Retorna:
    - DataFrame: Datos del archivo JSON en formato tabular.
    """
    with open(json_path, "r") as file:
        data = json.load(file)
        
    df = pd.json_normalize(data=data['entity'], 
                           meta=[['vehicle', 'trip', 'tripId'],
                                 ['vehicle', 'trip', 'startTime'],
                                 ['vehicle', 'trip', 'startDate'],
                                 ['vehicle', 'trip', 'routeId'],
                                 ['vehicle', 'trip', 'directionId'],
                                 ['vehicle', 'position', 'latitude'],
                                 ['vehicle', 'position', 'longitude'],
                                 ['vehicle', 'position', 'bearing'],
                                 ['vehicle', 'position', 'odometer'],
                                 ['vehicle', 'position', 'speed'],
                                 ['vehicle', 'timestamp'],
                                 ['vehicle', 'vehicle', 'licensePlate'],
                                 'id'],
                           errors='ignore')
    return df

df_wi = load_with_trip_update('/app/data/output/json_file/with_trip_update.json')
df_wo = load_without_trip_update('/app/data/output/json_file/without_trip_update.json')